<a href="https://colab.research.google.com/github/ochi06/portfolio/blob/main/pytorch%EF%BC%BFiris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()

#入力値と目標値を抽出
x = iris['data']
t = iris['target']

#PyTorchのTensor型へ変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

DataLoader

In [ ]:
#入力値と目標値をまとめる
dataset = torch.utils.data.TensorDataset(x,t)

データセット分割

In [ ]:
#各データのサンプル数を決定
#train : val : test = 60% : 20% : 20%
n_train =int(len(dataset)*0.6)
n_val = int(len(dataset)*0.2)
n_test = len(dataset)-n_train - n_val

torch.manual_seed(42)

#データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

ミニバッチ学習

In [ ]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test,batch_size)

x,t=next(iter(train_loader))

ネットワークの定義

4→4→3の全結合層を定義

In [ ]:
class Net(nn.Module):

#使用するオブジェクトを定義
  def __init__(self):
    super().__init__()
    self.fc1=nn.Linear(4,4)
    self.fc2=nn.Linear(4,3)

  #準伝播
  def forward(self,x):
    h=self.fc1(x)
    h=F.relu(h)
    h=self.fc2(h)
    return h

torch.manual_seed(0)

#インスタンス化
net=Net()

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

batch=next(iter(train_loader))

x,t =batch
#予測値の算出
y=net(x)


In [ ]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
x=x.to(device)
t=t.to(device)

torch.manual_seed(0)

#モデルのインスタンス化とデバイスへの転送
net=Net().to(device)

#最適化手法
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [ ]:
#エポック数
max_epoch=5

#学習のループ
for epoch in range(max_epoch):
  for batch in train_loader:
    x,t=batch
    x=x.to(device)
    t=t.to(device)
    y=net(x)
    loss=F.cross_entropy(y,t)
    print(f'loss:{loss}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

loss:1.0908641815185547
loss:1.0624275207519531
loss:1.0957728624343872
loss:1.0879180431365967
loss:1.0413137674331665
loss:1.0554883480072021
loss:1.0475170612335205
loss:1.0691304206848145
loss:1.0631484985351562
loss:1.016585111618042
loss:1.056710124015808
loss:1.0293322801589966
loss:1.0432616472244263
loss:1.0319433212280273
loss:1.0524427890777588
loss:0.9986320734024048
loss:1.010651707649231
loss:1.0500200986862183
loss:1.0783939361572266
loss:1.0198673009872437
loss:1.023457646369934
loss:1.0020211935043335
loss:1.053947925567627
loss:1.022329330444336
loss:1.0232975482940674


評価指標の追加

In [ ]:
x,t=next(iter(train_loader))
x=x.to(device)
t =t.to(device)
y=net(x)

y_label=torch.argmax(y, dim=1)

accuracy=(y_label==t).sum().float()/len(t)
accuracy

In [ ]:
#正解率を計算する関数
def calc_accuracy(data_loader):
  with torch.no_grad():
    total=0
    correct=0.0
    for batch in data_loader:
      x,t =batch
      x=x.to(device)
      t=t.to(device)
      y=net(x)
      y_label=torch.argmax(y, dim=1)
      total+=len(t)
      correct +=(y_label==t).sum()

    #全体の平均を算出
    accuracy=correct/total

  return accuracy

In [ ]:
calc_accuracy(train_loader)

In [ ]:
calc_accuracy(test_loader)